BRIAN WAHOME: N PUZZLE Solver Python3.

This is an N Puzzle Solver Using A* Search. It is a generalized version of the implementation by Milan Pecov(See bottom for reference. It takes in any N puzzle size as a list of lists and returns the steps take to reach the goal. The way the heuristics are used within the A* is slightly off but you can figure out a way to make them work for you depending on how you want to evaluate the cost to the goal state as generated. You can modify the goal generator to return any goal state you desire. You can implement as many heuristics as you want on condition that they return an integer. Add this to the list of functions(heuristics) in the main and make use of it wherever you please.)

Puzzle Node Class and necessary methods.
Note, my Eucledian function is honestly not the most optimum it could be but it does function and outperforms the manhattan and misplaced tiles heuristics due to Its sensitivity to the directions. It also gives the most accurate distances.

I never implemented the decorators extensions but have implemented the rest of the extensions listed.

In [83]:
class PuzzleNode:
    "Had this above init as it is independent of it and will be used there"

    @staticmethod
    def goalGenerator(state):
        # Goal State Generator
        n = len(state)
        r_count, row, goal, monitor, coordinates = 0, [], [], 1, {}
        for i in range(0, n ** 2, 1):
            row.append(i)
            coordinates[i] = (r_count, monitor - 1)
            if i == (((n) * monitor) - 1):
                goal.append(row)
                monitor += 1
                r_count, row = 0, []
                continue
            r_count += 1
        return goal, coordinates

    @staticmethod
    def h3(puzzle):
        """EUCLEDIAN DISTANCE"""
        """
        This will sum up the distance between an node and goal state
        using Its coordinates and goal state coordinates.
        This basically takes the shortest distance from points to goals and sums up the distances.
        """
        generated_coordinates = PuzzleNode.goalGenerator(puzzle)[1]
        cood_values = [vals for vals in generated_coordinates.values()]
        puzzleMap, oneList = {}, []
        for rows in puzzle:
            for els in rows:
                oneList.append(els)
        for nav in range(len(oneList)):  # Map puzzle to coordinates
            puzzleMap[oneList[nav]] = cood_values[nav]
        # Now generate differences
        eucledian = 0
        for value in oneList:
            eucledian += ((puzzleMap[value][0] - generated_coordinates[value][0]) ** 2 +
                         (puzzleMap[value][0] - generated_coordinates[value][0]) ** 2 +
                         (puzzleMap[value][1] - generated_coordinates[value][1]) ** 2 +
                         (puzzleMap[value][1] - generated_coordinates[value][1]) ** 2) ** 0.5
        return eucledian

    @staticmethod
    def h2(puzz):
        """ Misplaced Tiles"""
        manhattan = 0
        m = eval(str(puzz))
        for i in range(len(m)):
            for j in range(len(m)):
                if m[i][j] == 0: continue
                manhattan += abs(i - (m[i][j] / len(m))) + abs(j - (m[i][j] % len(m)));
        return manhattan

    def h1(puzzle):
        """Manhattan distance"""
        manhattan = 0
        matrix = eval(str(puzzle))
        for i in range(len(matrix)):
            for j in range(len(matrix)):
                if matrix[i][j] == 0: continue
                manhattan += abs(i - (matrix[i][j] / len(matrix))) + abs(j - (matrix[i][j] % len(matrix)));
        return manhattan

    def __init__(self, puzzleState):
        self.puzzleState = puzzleState  # Using a rowwise list of blocks
        self.n = len(puzzleState)  # Puzzle Dimension
        self.goal = PuzzleNode.goalGenerator(eval(str(puzzleState)))[0]

    "Puzzle printer"

    def __str__(self):
        # PRINT Implementation
        print("*************CURRENT STATE******************")
        mTabs = "{}".format(" " * 17)
        for rows in self.puzzleState:
            print("{}".format(mTabs) +
                  "|" + "|".join([str(x) for x in rows]) + "|" + "{}".format(mTabs))
        return ""

    # Solvability checker for 8 puzzle: Counts the number of inversion.
    # If inversions%2==0? Solvable, else false.
    # Will be called in the puzzleSolver.
    def solvable(self):
        # Create one array from this
        oneList = []
        for rows in puzzle:
            for els in rows:
                oneList.append(els)
        inversions = 0
        oneList.remove(0)
        for f in range(len(oneList)):
            for b in range(len(oneList[:f:-1])):
                if (oneList[f] > oneList[::-1][b]):
                    inversions += 1

        solvable = not (inversions % 2 == 1)
        return solvable

    "Validates that the puzzle is a 'square', has unique elements which are < (n^2) -1"

    def puzzleValidator(self):  # Validation step(14/15)
        # Validate all elements are unique and valid: Uniqueness test
        validPuzzleElements = {}
        for i in range(0, len(self.puzzleState) ** 2, 1):
            validPuzzleElements[str(i)] = 1
        for row in self.puzzleState:
            for elements in row:
                try:
                    validPuzzleElements[str(elements)]
                except:
                    break
                validPuzzleElements[str(elements)] -= 1

        lengthCheck, uniqueCheck = True, True
        for val in validPuzzleElements.values():
            if val != 0:
                lengthCheck = False
            if val < 0:
                uniqueCheck = False
        return lengthCheck and uniqueCheck

    def next_Moves(self, puzzle):
        """
        Returns a list of all possible next_Moves
        """
        nextMoves = []

        matrix = eval(str(puzzle))
        i = 0
        while 0 not in matrix[i]: i += 1
        j = matrix[i].index(0);  # blank space (zero)

        if i > 0:
            matrix[i][j], matrix[i - 1][j] = matrix[i - 1][j], matrix[i][j];  # move up
            nextMoves.append(str(matrix))
            matrix[i][j], matrix[i - 1][j] = matrix[i - 1][j], matrix[i][j];

        if i < len(matrix) - 1:
            matrix[i][j], matrix[i + 1][j] = matrix[i + 1][j], matrix[i][j]  # move down
            nextMoves.append(str(matrix))
            matrix[i][j], matrix[i + 1][j] = matrix[i + 1][j], matrix[i][j]

        if j > 0:
            matrix[i][j], matrix[i][j - 1] = matrix[i][j - 1], matrix[i][j]  # move left
            nextMoves.append(str(matrix))
            matrix[i][j], matrix[i][j - 1] = matrix[i][j - 1], matrix[i][j]

        if j < len(matrix) - 1:
            matrix[i][j], matrix[i][j + 1] = matrix[i][j + 1], matrix[i][j]  # move right
            nextMoves.append(str(matrix))
            matrix[i][j], matrix[i][j + 1] = matrix[i][j + 1], matrix[i][j]

        return nextMoves

    "The solve puzzle function."

    def solvePuzzle(self, n, state, heuristic, prnt=False):
        # Print function (8)
        print("Started")

        # Validation step(14/15)
        if not self.puzzleValidator():
            return 0, 0, -1
        # Extension[3.1]
        # Validation for 8 Puzzles(The inversion checker was specialized only for 8)
        if len(puzzle) == 3:
            solvable = self.solvable()
            if not solvable:
                print("This is an unsolvable configuration")
                return 0, 0, -2

        moves = self.next_Moves
        # A* algorithm
        end = self.goal
        front = [[heuristic(state), state]]  # optional: heuristic_1
        frontier = []
        frontierSize = 0
        while 1:
            i = 0
            for j in range(1, len(front)):
                if front[i][0] > front[j][0]:
                    i = j
            path = front[i]
            front = front[:i] + front[i + 1:]
            endnode = path[-1]
            if str(endnode) == str(end):
                break
            if endnode in frontier: continue
            for k in self.next_Moves(endnode):
                if k in frontier: continue
                newpath = [path[0] + heuristic(k) - heuristic(endnode)] + path[1:] + [k]
                front.append(newpath)
                frontier.append(endnode)
            frontierSize += 1
        if prnt:
            print(self)
            print("PATHS TO GOAL")
            state = 1
            mTabs = "{}".format(" " * 17)
            for states in path[1:]:
                print("STATE:", state)
                for rows in eval(str(states)):
                    print("{}".format(mTabs) +
                          "|" + "|".join([str(x) for x in rows]) + "|" +
                          "{}".format(mTabs))
                state += 1
            print("GOAL ACHIEVED")
            print("NUMBER OF MOVES: {},{} MAXIMUM FRONTIER SIZE: {}".format(len(path), '\n', frontierSize))
        return frontierSize, path, 0


heuristics = [PuzzleNode.h1, PuzzleNode.h2, PuzzleNode.h3]
"""
puzzle = [[1, 2, 6, 3,16,17], [18,19,4, 9, 5, 7], 
[8,20,21, 13, 11, 15], [12, 14,22,23, 0, 10],
[24,26,28,31,32,34],[35,33,27,29,25,30]]
"""
puzzle = [[7, 2, 4], [5, 0, 6], [8, 3, 1]]
# puzzle = [[0, 1,2], [3, 4, 5], [6,8, 7]]
# puzzle = [[8, 1,2], [0, 4, 3], [7,6, 5]]

puzzle1 = PuzzleNode(puzzle)
puzzle1.solvePuzzle(4, puzzle, heuristics[1], True)

Started
*************CURRENT STATE******************
                 |7|2|4|                 
                 |5|0|6|                 
                 |8|3|1|                 

PATHS TO GOAL
STATE: 1
                 |7|2|4|                 
                 |5|0|6|                 
                 |8|3|1|                 
STATE: 2
                 |7|2|4|                 
                 |5|6|0|                 
                 |8|3|1|                 
STATE: 3
                 |7|2|0|                 
                 |5|6|4|                 
                 |8|3|1|                 
STATE: 4
                 |7|0|2|                 
                 |5|6|4|                 
                 |8|3|1|                 
STATE: 5
                 |0|7|2|                 
                 |5|6|4|                 
                 |8|3|1|                 
STATE: 6
                 |5|7|2|                 
                 |0|6|4|                 
                 |8|3|1|              

(104,
 [3.000000000000009,
  [[7, 2, 4], [5, 0, 6], [8, 3, 1]],
  '[[7, 2, 4], [5, 6, 0], [8, 3, 1]]',
  '[[7, 2, 0], [5, 6, 4], [8, 3, 1]]',
  '[[7, 0, 2], [5, 6, 4], [8, 3, 1]]',
  '[[0, 7, 2], [5, 6, 4], [8, 3, 1]]',
  '[[5, 7, 2], [0, 6, 4], [8, 3, 1]]',
  '[[5, 7, 2], [6, 0, 4], [8, 3, 1]]',
  '[[5, 7, 2], [6, 4, 0], [8, 3, 1]]',
  '[[5, 7, 2], [6, 4, 1], [8, 3, 0]]',
  '[[5, 7, 2], [6, 4, 1], [8, 0, 3]]',
  '[[5, 7, 2], [6, 4, 1], [0, 8, 3]]',
  '[[5, 7, 2], [0, 4, 1], [6, 8, 3]]',
  '[[5, 7, 2], [4, 0, 1], [6, 8, 3]]',
  '[[5, 7, 2], [4, 1, 0], [6, 8, 3]]',
  '[[5, 7, 2], [4, 1, 3], [6, 8, 0]]',
  '[[5, 7, 2], [4, 1, 3], [6, 0, 8]]',
  '[[5, 7, 2], [4, 0, 3], [6, 1, 8]]',
  '[[5, 7, 2], [4, 3, 0], [6, 1, 8]]',
  '[[5, 7, 0], [4, 3, 2], [6, 1, 8]]',
  '[[5, 0, 7], [4, 3, 2], [6, 1, 8]]',
  '[[0, 5, 7], [4, 3, 2], [6, 1, 8]]',
  '[[4, 5, 7], [0, 3, 2], [6, 1, 8]]',
  '[[4, 5, 7], [3, 0, 2], [6, 1, 8]]',
  '[[4, 0, 7], [3, 5, 2], [6, 1, 8]]',
  '[[4, 7, 0], [3, 5, 2], [6, 1, 8]]',


Test Cases.

In [80]:
test1 = [[5,7,6],[2,4,3],[8,1,0]]
test2 = [[7,0,8],[4,6,1],[5,3,2]]
test3 = [[2,3,7],[1,8,0],[6,5,4]]
#test4=[[3,9,13,14],[4,7,12,15],[2,5,8,11],[1,6,10,0]]

In [81]:
heuristics = [PuzzleNode.h1, PuzzleNode.h2, PuzzleNode.h3]
print("RUNNING TESTS.")
puzzle = test1
Puzzle = PuzzleNode(puzzle)
Puzzle.solvePuzzle(3, puzzle, heuristics[0],True)
print("TEST 1 DONE")
print("*********************************************************************************************************")
print("TEST 2 STARTING")

puzzle = test2
Puzzle = PuzzleNode(puzzle)
Puzzle.solvePuzzle(3, puzzle, heuristics[0],True)
print("TEST 2 DONE")
print("*********************************************************************************************************")
print("TEST 3 STARTING")
puzzle = test3
Puzzle = PuzzleNode(puzzle)
Puzzle.solvePuzzle(3, puzzle, heuristics[0],True)
print("TEST 3 DONE")
print("*********************************************************************************************************")
#print("TEST 4 STARTING(A 4X4)")
#puzzle = test4
#Puzzle = PuzzleNode(puzzle)
#Puzzle.solvePuzzle(3, puzzle, heuristics[0],True)

print("FINISHED")

RUNNING TESTS.
Started
*************CURRENT STATE******************
                 |5|7|6|                 
                 |2|4|3|                 
                 |8|1|0|                 

PATHS TO GOAL
STATE: 1
                 |5|7|6|                 
                 |2|4|3|                 
                 |8|1|0|                 
STATE: 2
                 |5|7|6|                 
                 |2|4|0|                 
                 |8|1|3|                 
STATE: 3
                 |5|7|6|                 
                 |2|4|3|                 
                 |8|1|0|                 
STATE: 4
                 |5|7|6|                 
                 |2|4|3|                 
                 |8|0|1|                 
STATE: 5
                 |5|7|6|                 
                 |2|4|3|                 
                 |0|8|1|                 
STATE: 6
                 |5|7|6|                 
                 |0|4|3|                 
                 |2|8|1

The results obtained were as follows:
For the list
<table>
<tr><td><b>[[5,7,6],[2,4,3],[8,1,0]]</b></td><td><ul><li><b>NUMBER OF MOVES:48</b></li><li>MAXIMUM FRONTIER SIZE:258</li></ul></td></tr>
<tr><td><b>[[7,0,8],[4,6,1],[5,3,2]]</b></td><td><ul><li><b>NUMBER OF MOVES:61</b></li><li>MAXIMUM FRONTIER SIZE:307</li></ul></td></tr>
<tr><td><b>[[2,3,7],[1,8,0],[6,5,4]]</b></td><td><ul><li><b>NUMBER OF MOVES:71</b></li><li>MAXIMUM FRONTIER SIZE:326</li></ul></td></tr>

</table>

ACKNOWLEDGEMENT:

BFS implementation 15 puzzle:
Milan Pecov 15 puzzle solution: 
https://raw.githubusercontent.com/MilanPecov/15-Puzzle-Solvers/master/main.py

Inversions for 8 puzzle testing logic
https://www.geeksforgeeks.org/check-instance-8-puzzle-solvable/
